In [1]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf
from statsmodels.tsa.stattools import adfuller
import statsmodels.formula.api as smf
from itertools import product
from tqdm import tqdm_notebook
import warnings
warnings.filterwarnings("ignore")
sns.set_theme()

## 4. **CO2 emissions in the future**

Up until now we have looked at the current levels of CO2 emissions. Now it is difficult to predict how ambitious the climate actions of different countries will be. One unpleasent scenario we *can* to some extent predict the implications of is: Status Quo.

So if we assume status quo regarding factors that contribute to the rise in CO2 emissions what does the future then look like?

We investigate this by considering the CO2 emissions data as a time series. As seen from the plot of CO2 emissions below the data seems to have an exponentially increasing trend and an increasing variance over time.

In [2]:
df3 = pd.read_csv('data/annual-co-emissions-by-region.csv')
df3 = df3.rename(columns={'Annual CO2 emissions (zero filled)': "Annual CO2 emissions"})
df3 = df3.loc[df3['Entity'] == 'World']
df3 = df3.sort_values('Year')
diff1 = np.diff(np.log(df3['Annual CO2 emissions'])) 
df3['Log transformed first order differenced CO2 emissions'] = np.pad(diff1, (0, 1), 'constant')
df3['Log transformed CO2 emissions'] = np.log(df3['Annual CO2 emissions'])
model_arima = ARIMA(df3['Log transformed CO2 emissions'], order=(1,1,2))
model_fit = model_arima.fit()
y = 2020
for i in range(10):
    y = y + 1
    df3 = df3.append({'Entity': 'World', 'Code': 'OWID_WRL','Year': y, 'Annual CO2 emissions': float("NAN"), 'Log transformed first order differenced CO2 emissions': float("NAN"),'Log transformed CO2 emissions': float("NAN")}, ignore_index=True)
preds = model_fit.get_prediction(0,281)  # 95% conf
preds_ci = preds.conf_int()
preds_mu = preds.predicted_mean
df3['Predictions'] = preds_mu
df3['CI_lower'] = preds_ci[1:]['lower Log transformed CO2 emissions'].reset_index(drop=True)
df3['CI_upper'] = preds_ci[1:]['upper Log transformed CO2 emissions'].reset_index(drop=True)

In [3]:
fig = px.line(df3, x='Year', y='Annual CO2 emissions', title = 'Annual CO2 emissions')
fig.show()

To model the expontential trend in CO2 emissions, the variance and ensure stationarity we chose to model the log-transformed CO2 emissions data with an autoregressive integrated moving average model (ARIMA). To determine the order of the parameters in the ARIMA model we performed a grid search of all parameter combinations in the range of 0 to 7 where the parameter combination with the lowest AIC was chosen. The final model ended up being an ARIMA(1,1,2). I.e. according to the model, next years CO2 emissions is a combination of this year's emissions + an upward trend + the errors the model made this and the previous year.

Below you can investigate the predictions of the ARIMA-model until 2030. The model predicts a small decrease in CO2 emissions in 2021 follow by an upward linear increase for the next 9 years.

In [4]:
# Plot 
fig = px.line(df3, x='Year', y='Annual CO2 emissions', range_x = [1920,2030], title="ARIMA predictions of annual CO2 emissions")
fig.add_scatter(x=df3['Year'], y=df3['Annual CO2 emissions'], mode='lines', name = 'Original data')
fig.add_scatter(x=df3['Year'], y=np.exp(df3['Predictions']), mode='lines', name = 'Predictions')
fig.add_traces([go.Scatter(x = df3['Year'], y = np.exp(df3['CI_upper']),
                           mode = 'lines', line_color = 'rgba(0,0,0,0)',
                           showlegend = False),
                go.Scatter(x = df3['Year'], y = np.exp(df3['CI_lower']),
                           mode = 'lines', line_color = 'rgba(0,0,0,0)',
                           name = '95% confidence interval',
                           fill='tonexty', fillcolor = 'rgba(0,100,80,0.2)')])
fig.show()

### How about the global temperature?

Well, we have just seen that assuming status quo and that CO2 emissions can be modeled by an ARIMA, the CO2 emission will increase further. We have also seen that the average global temperature and cumulative CO2 emissions have an approximately linear relationship.

Now, how would much will the global temperature have increased in 2030 if do nothing more than now about it?

This is investigated by using the ARIMA predictions of the CO2-level into the temperature-co2 model as fitted in the 1st section.

(Note: Making predictions which themselves are uncertain by nature based on unceratin input of the CO2 level, propogates a lot of uncertainty on these final predictions. This uncertainty has not yet been fully modelled, only the confidence intervals of the fit is illustrated below, i.e. the confidence intervals of the mean global temperature conditioned on the CO2 predictions from above).

In [5]:
# Loading the data & pre-processing
df = pd.read_csv('data/GlobalLandTemperaturesByCity.csv')
df['dt'] = pd.DatetimeIndex(df['dt'])
df['Year'] = pd.DatetimeIndex(df['dt']).year
df = df[df['Year'] != 1743]
df = df[df['Year'] != 2013]

# For each country average all AverageTemperature values and add iso_alpha to countries
df_avg = pd.DataFrame({'AverageTemperature': df.groupby('Country')['AverageTemperature'].mean(), 'Country': df.groupby('Country')['Country'].first()})
df_avg = df_avg.reset_index(drop=True)
#print('Average country temperature df: ', df_avg)

df_avg_city = pd.DataFrame({'AverageTemperature': df.groupby('City')['AverageTemperature'].mean(), 'City': df.groupby('City')['City'].first()})
df_avg_city = df_avg_city.reset_index(drop=True)
#print('\n \n Average city temperature df: ', df_avg_city)

# For each Country for each Year average all AverageTemperature values
D1 = df.groupby(['Year', 'Country'])['AverageTemperature'].mean().reset_index()
meantemp = D1.groupby('Year')['AverageTemperature'].mean().reset_index() # Rough - er ren average.
#meantemp1900 = meantemp[156:] # 1900-2012
#meantemp1900 = pd.read_csv('')
meantemp1900 = pd.read_csv('data/meantemprelative1900df.csv')
meantemp1900 = meantemp1900.drop(columns = {'Unnamed: 0'})
meantemp1900

df4 = pd.read_csv('data/annual-co-emissions-by-region.csv')
df4 = df4.rename(columns={'Annual CO2 emissions (zero filled)': "Annual CO2 emissions (tons)"})
df4 = df4.loc[df4['Entity'] == 'World']
df4 = df4.sort_values('Year')
meantemp1900co2 = pd.merge(meantemp1900, df4, left_on="Year", right_on="Year", how='inner')
meantemp1900co2 = meantemp1900co2.rename(columns={'AverageTemperature': "Average Temperature (C)"})

In [6]:
y = 2020
for i in range(10):
    y = y + 1
    df4 = df4.append({'Entity': 'World', 'Code': 'OWID_WRL','Year': y, 'Annual CO2 emissions (tons)': np.exp(np.array(df3[df3.Year == y]['Predictions'])[0])}, ignore_index=True)
    
df4['Annual CO2 emissions (tons)'] = np.cumsum(df4['Annual CO2 emissions (tons)'])
df4 = df4.rename(columns={"Annual CO2 emissions (tons)": "Cumulative CO2 emissions (tons)"})

In [7]:
# Predicting with the linear model
co2_predict = np.zeros(18)
c = 0
for i in range(2013,2031):
    co2_predict[c] = df4[df4.Year == i]['Cumulative CO2 emissions (tons)'].values[0]
    c += 1

In [8]:
def linear_fit(x):
    return 9.40088e-13 * x + 15.1500

In [9]:
pred_temp = linear_fit(co2_predict)

In [10]:
# Adding predicted values to dataframe
index = 113
year = 2014
df_pred = pd.DataFrame({'Cumulative CO2 emissions (tons)': co2_predict[0], 'Average Temperature (C)': pred_temp[0], 'Entity': 'World', 'Code': 'OWID_WRL', 'Year': year-1}, index=[0])

for i in range(1,18):
    df_pred = df_pred.append({'Cumulative CO2 emissions (tons)': co2_predict[i], 'Average Temperature (C)': pred_temp[i], 'Entity': 'World', 'Code': 'OWID_WRL', 'Year': year}, ignore_index=True)
    year += 1
    index += 1


In [11]:
meantemp1900co2 = meantemp1900co2.rename(columns={"Annual CO2 emissions (tons)": "Cumulative CO2 emissions (tons)"})
meantemp1900co2["Cumulative CO2 emissions (tons)"] = np.cumsum(meantemp1900co2["Cumulative CO2 emissions (tons)"])
df_predictions = pd.concat([meantemp1900co2, df_pred], ignore_index=True)

In [12]:
df_predictions2 = df_predictions.copy()
df_predictions2 = df_predictions2.rename(columns={"Average Temperature (C)": "temp", "Cumulative CO2 emissions (tons)": "CO2"})
model = smf.ols('temp ~ CO2', df_predictions2)
results = model.fit()
alpha = .05
predictions = results.get_prediction(df_predictions2).summary_frame(alpha)

In [13]:
fig = px.scatter(df_predictions, x="Cumulative CO2 emissions (tons)", y="Average Temperature (C)",
    color="Year", color_continuous_scale="balance",
    trendline="ols", trendline_color_override="black", title="Global temperature vs. cumulated CO2")

ci_upper = predictions['mean_ci_upper']
ci_lower = predictions['mean_ci_lower']

fig.add_traces([go.Scatter(x = df_predictions["Cumulative CO2 emissions (tons)"], y = ci_upper,
                           mode = 'lines', line_color = 'rgba(0,0,0,0)',
                           showlegend = False),
                go.Scatter(x = df_predictions["Cumulative CO2 emissions (tons)"], y = ci_lower,
                           mode = 'lines', line_color = 'rgba(0,0,0,0)',
                           name = '95% confidence interval',
                           fill='tonexty', fillcolor = 'rgba(0,100,80,0.2)')])

fig.show()

Clearly the global temperature is estimated to increase *a lot* - if we just continue bussiness-as-usual.

Note that the CO2 levels from 2012 to 2020 are actual measurements. The CO2 levels from 2021 to 2030 are predicted by the ARIMA-model.

According to the models and data, the global temperature is expected to increase from **16.4** in 2012 to **17.1** in 2030, an *additional* increase of **0.7** degress in less than two centuries. In conclussion, now is the time for climate action.

Sources: http://web.mit.edu/jlcohen/www/papers/AML-WhitePaper.pdf